### read data

In [14]:
import pandas as pd

fd=pd.read_excel("./大病新表.xlsx",sheet_name="Sheet2")
# fd.fillna(0, inplace=True)
fd.columns
# fd.dtypes
# fd.set_index(pd.to_datetime(fd['借支日期']),inplace=True)
# fd['借支日期']=(pd.to_datetime(fd['借支日期'],format='%Y%m%d'))
# fd['xi借支日期'] = fd['借支日期'].astype(str)
# fd['借支日期'] = pd.to_datetime(fd['借支日期'], format='%Y%m%d.0', errors='coerce')

# fd['借支日期'] = fd['借支日期'].astype('datetime64[ns]')
# fd.head(20)
# print("读取<<大病新表>>成功！")
# print("共计{}条记录！".format(fd.shape[0]))

Index(['科室', '住院号', '姓名', '借支日期', '借支金额', '还款日期', '还款金额', '还款日期.1', '还款金额.1',
       '还款日期.2', '还款金额.2', '批次'],
      dtype='object')

In [15]:
fd[fd['姓名']=='宋淑珍']

,科室,住院号,姓名,借支日期,借支金额,还款日期,还款金额,还款日期.1,还款金额.1,还款日期.2,还款金额.2,批次
222,乳甲外,828174,宋淑珍,20200730.0,17388.69,20200730.0,13848.4,20200730.0,3000.0,NaN,NaN,NaN
301,乳甲外,828174,宋淑珍,20200814.0,2892.11,20200814.0,2140.07,NaN,NaN,NaN,NaN,NaN
467,乳甲外,828174,宋淑珍,20200914.0,2949.20,20200914.0,2582.16,NaN,NaN,NaN,NaN,NaN
589,乳甲外,828174,宋淑珍,20200929.0,2602.48,20200929.0,2262.41,NaN,NaN,NaN,NaN,NaN
665,乳甲外,828174,宋淑珍,20201012.0,2467.62,20201012.0,2094.97,NaN,NaN,NaN,NaN,NaN


In [ ]:
fd.describe()

1. filter by fd["借支金额"].isna()

In [ ]:
fd[fd["借支金额"].isna()]

1.1 continue to filter fd["借支金额"].isna()&fd["还款金额"].notna()


fd[fd["借支金额"].isna()&fd["还款金额"].notna()&fd["还款金额.1"].isna()]

fd[fd["借支金额"].isna()&fd["还款金额"].notna()&fd["还款金额.1"].notna()]


2. filter by fd["借支金额"].notna()

fd[fd["借支金额"].notna()]

### testing a filter

In [ ]:
target=pd.read_excel("./base.xls",sheet_name="Sheet1")
target.fillna(0, inplace=True)
# target[['借支金额']].fillna(0, inplace=True)
# target[['还款金额','借支金额']]=target[['还款金额','借支金额']].fillna(0, inplace=True)
# target.info()
target.columns

In [ ]:
import process,time
result = pd.DataFrame(columns = fd.columns) # 错误结果 
matched =  pd.DataFrame(columns = fd.columns)  # 正确结果
drop_index=[] #记录需要删除对行号

max_steps=target.shape[0]
process_bar = process.ShowProcess(max_steps, 'OK')
for idx,row in target.iterrows():
    # print(type(row))
    # print(row["住院号"])
    fil=(fd["住院号"]==row["住院号"]) \
        &(fd["科室"]==row["科室"]) \
        &(fd["姓名"]==row["姓名"]) \
        &(fd["借支金额"]==row["借支金额"]) \
        &(fd["还款金额"]==row["还款金额"]) \
        &(fd["还款金额.1"]==row["还款金额.1"]) \
        &(fd["还款金额.2"]==row["还款金额.2"]) 
    _temp= fd[fil]  
    if len(_temp)==0: # can not match
        # print("{} is false".format(idx))
        result.loc[len(result)] = row
    else:
        # print("{} is true,fd index is {}".format(idx,_temp.index.tolist()))
        matched.loc[len(matched)] = row
        drop_index.extend(_temp.index.tolist())
    process_bar.show_process()
    time.sleep(0.01)
# print(result)
# write to error
result.to_excel("./result.xlsx",sheet_name="Sheet1",index=False)
matched.to_excel("./matched.xlsx",sheet_name="Sheet1",index=False)
# print(drop_index)
left=fd.drop(drop_index) # 剩余对比结果
left.to_excel("./left.xlsx",sheet_name="Sheet1",index=False)
print("对比结果如下：")
print("-----------------------------------")
print("一共对比记录："+str(target.shape[0]))
print("其中，正确比对合计："+str(matched.shape[0]))
print("错误对比结果合计："+str(result.shape[0]))
print("原始结果合计："+str(fd.shape[0]))
print("剩余对比结果合计："+str(left.shape[0]))
print("-----------------------------------")
print("最后产生三个文件：")
print("错误结果：result.xlsx")
print("正确结果：matched.xlsx")
print("剩余结果：left.xlsx")
print("-----------------------------------")

# left.shape

### 例子： convert data to datetime

In [ ]:
import pandas as pd

df=pd.read_excel("./base.xls",sheet_name="Sheet1")
# df.head(10)
# fd.dtypes
# fd.set_index(pd.to_datetime(fd['借支日期']),inplace=True)
# fd['借支日期']=(pd.to_datetime(fd['借支日期'],format='%Y%m%d'))
# fd['xi借支日期'] = fd['借支日期'].astype(str)
df['还款日期'] = pd.to_datetime(df['还款日期'].astype(str), format='%Y%m%d.0', errors='coerce')
df['借支日期'] = pd.to_datetime(df['借支日期'].astype(str), format='%Y%m%d.0', errors='coerce')
df
# fd['借支日期'] = fd['借支日期'].astype('datetime64[ns]')
# fd.head(20)

In [ ]:
import pandas as pd
pd.to_datetime(pd.Series([20100329.0, 20100328.0, 20100329.0]).astype(str), format='%Y%m%d.0')

In [ ]:
import numpy as np
target=pd.read_excel("./base.xls",sheet_name="Sheet1")
target.fillna(0, inplace=True)
target.replace(0,np.nan,inplace=True)
target.head(10)
target.to_excel("./test.xlsx",sheet_name="Sheet1",index=False)